In [7]:
from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting.lib import crossover
import yfinance as yf
# import pandas_ta as ta
import talib

df = yf.download('GOOG','2020-1-1','2024-4-30', auto_adjust=True)
#Check if NA values are in data
df=df[df['Volume']!=0]
df.isna().sum()
print(df.head)

[*********************100%%**********************]  1 of 1 completed

<bound method NDFrame.head of                   Open        High         Low       Close    Volume
Date                                                                
2020-01-02   67.077499   68.406998   67.077499   68.368500  28132000
2020-01-03   67.392998   68.625000   67.277199   68.032997  23728000
2020-01-06   67.500000   69.824997   67.500000   69.710503  34646000
2020-01-07   69.897003   70.149498   69.518997   69.667000  30054000
2020-01-08   69.603996   70.579002   69.542000   70.216003  30560000
...                ...         ...         ...         ...       ...
2024-04-23  158.589996  160.479996  157.964996  159.919998  16115400
2024-04-24  159.089996  161.389999  158.820007  161.100006  19485700
2024-04-25  153.360001  158.279999  152.768005  157.949997  36197800
2024-04-26  175.990005  176.419998  171.399994  173.690002  56500800
2024-04-29  170.770004  171.380005  167.059998  167.899994  35914600

[1088 rows x 5 columns]>


In [8]:
def optim_func(series):
    if series['# Trades'] < 10:
        return -1

    return series['Equity Final [$]'] / series['Exposure Time [%]']

class RsiOscillator(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14
    
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        
    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close() # sell 
        elif crossover(self.lower_bound, self.rsi):
            self.buy() # buy


In [9]:
bt = Backtest(df, RsiOscillator, cash = 10_000)
stats = bt.run()
# stats = bt.optimize(
#     upper_bound = range(50, 85, 5),
#     lower_bound = range(10, 45, 5),
#     rsi_window = range(10, 30, 2),
#     maximize = 'Sharpe Ratio'
#     constraint = lambda param: param.upper_bound > param.lower_bound,
#     max_tries = 100
# )

# stats = bt.optimize(
#     upper_bound = range(50, 85, 5),
#     lower_bound = range(10, 45, 5),
#     rsi_window = range(10, 30, 2),
#     maximize = optim_func
# )
stats

Start                     2020-01-02 00:00:00
End                       2024-04-29 00:00:00
Duration                   1579 days 00:00:00
Exposure Time [%]                   34.834559
Equity Final [$]                  10233.49939
Equity Peak [$]                  15179.121979
Return [%]                           2.334994
Buy & Hold Return [%]              145.580925
Return (Ann.) [%]                     0.53604
Volatility (Ann.) [%]               24.919964
Sharpe Ratio                          0.02151
Sortino Ratio                        0.031483
Calmar Ratio                         0.012359
Max. Drawdown [%]                  -43.373676
Avg. Drawdown [%]                   -7.158887
Max. Drawdown Duration      817 days 00:00:00
Avg. Drawdown Duration       76 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                      29.446572
Worst Trade [%]                    -21.009299
Avg. Trade [%]                    

In [10]:
bt.plot()

/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.ac

GridPlot(id='p1732', ...)